In [1]:
import os
import h5py
import pathlib
import logging
import numpy as np
import pandas as pd

import tensorflow
if int(tensorflow.__version__.split(".")[0]) >= 2:
    import tensorflow.compat.v1 as tf
else:
    import tensorflow as tf

from datetime import datetime
import multiprocessing
import argparse
import json
import pathlib
# from tqdm import tqdm
import sys
# sys.path.append(pathlib.Path(__file__).parent.absolute())
from commons import input_iterator

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
from commons import cnn_bi_lstm_model

os.chdir('/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/')

In [2]:
parser = argparse.ArgumentParser(description='Argument parser for generating model predictions.')
optional_arguments = parser._action_groups.pop()
required_arguments = parser.add_argument_group('required arguments')
required_arguments.add_argument('--pre-processed-dir', help='Pre-processed data directory', default='/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/preprocessed_new', required=False)

optional_arguments.add_argument('--model', help='Pre-trained prediction model name (default: CHAP_ALL_ADULTS)', default='CHAP_ALL_ADULTS',
    required=False, choices=['CHAP_A', 'CHAP_B', 'CHAP_C', 'CHAP', 'CHAP_ALL_ADULTS', 'CHAP_CHILDREN', 'CHAP_AUSDIAB'])
optional_arguments.add_argument('--predictions-dir', help='Predictions output directory (default: ./predictions)', default='./predictions', required=False) 
optional_arguments.add_argument('--no-segment', help='Do not output segment number', default=False, required=False, action='store_true')
optional_arguments.add_argument('--output-label', help='Whether to output the actual label', default=False, required=False, action='store_true')

optional_arguments.add_argument('--model-checkpoint-path', help='Path where the custom trained model checkpoint is located', default=None, required=False)
optional_arguments.add_argument('--cnn-window-size', help='CNN window size of the model in seconds on which the predictions to be made (default: 10).', default=10, type=int, required=False)
optional_arguments.add_argument('--bi-lstm-window-size', help='BiLSTM window size in minutes (default: 7).', default=None, required=False, type=int)
optional_arguments.add_argument('--down-sample-frequency', help='Downsample frequency in Hz for GT3X data (default: 10).', default=10, type=int, required=False)   
optional_arguments.add_argument('--gt3x-frequency', help='GT3X device frequency in Hz (default: 30)', default=30, type=int, required=False)
optional_arguments.add_argument('--activpal-label-map', help='ActivPal label vocabulary (default: {"sitting": 0, "not-sitting": 1, "no-label": -1})', default='{"sitting": 0, "not-sitting": 1, "no-label": -1}', required=False)
optional_arguments.add_argument('--silent', help='Whether to hide info messages', default=False, required=False, action='store_true')
optional_arguments.add_argument(
    '--padding',
    type=str,
    help='Padding scheme for the last part of data that does not fill a whole lstm window (default: %(default)s)',
    default='drop',
    choices=('drop', 'zero', 'wrap')
)
parser._action_groups.append(optional_arguments)
args = parser.parse_known_args()
args = args[0]

if not os.path.exists(args.predictions_dir):
    os.makedirs(args.predictions_dir)

label_map = json.loads(args.activpal_label_map)
label_map = {label_map[k]:k for k in label_map}

bi_lstm_window_sizes = {"CHAP_A": 9, "CHAP_B": 9, "CHAP_C": 7, "CHAP_ALL_ADULTS": 7, "CHAP_CHILDREN": 3, "CHAP_AUSDIAB": 9}
if args.bi_lstm_window_size is None:
    bi_lstm_window_sizes['CUSTOM_MODEL'] = 7
else:
    bi_lstm_window_sizes[args.model] = args.bi_lstm_window_size


    bi_lstm_window_sizes['CUSTOM_MODEL'] = args.bi_lstm_window_size

if args.model_checkpoint_path is not None:
    if not args.silent:
        print('Loading custom model from checkpoint path: {}'.format(args.model_checkpoint_path))
    args.model = 'CUSTOM_MODEL'
else:
    args.model_checkpoint_path =  '/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/pre-trained-models'

pre_processed_data_dir = args.pre_processed_dir
output_dir=args.predictions_dir
model=args.model
segment=not args.no_segment
output_label=args.output_label

label_map=label_map
downsample_window=1.0/args.down_sample_frequency
cnn_window_size=args.cnn_window_size
gt3x_frequency=args.gt3x_frequency
model_ckpt_path=args.model_checkpoint_path
padding=args.padding

In [3]:
tf.reset_default_graph()
p = max(1, multiprocessing.cpu_count()//2)
sess = tf.Session(config=tf.ConfigProto(inter_op_parallelism_threads=p, intra_op_parallelism_threads=p))
tf.saved_model.loader.load(sess, ["serve"], os.path.join(model_ckpt_path, "{}".format(model)))


2024-05-06 12:01:47.583195: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2024-05-06 12:01:47.615831: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2799925000 Hz
2024-05-06 12:01:47.616280: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559f9f77d2b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-05-06 12:01:47.616450: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-05-06 12:01:47.617635: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


2024-05-06 12:01:49.011960: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-05-06 12:01:49.012017: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (animeshkumar-msi): /proc/driver/nvidia/version does not exist
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


INFO:tensorflow:Restoring parameters from /home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/pre-trained-models/CHAP_ALL_ADULTS/variables/variables


INFO:tensorflow:Restoring parameters from /home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/pre-trained-models/CHAP_ALL_ADULTS/variables/variables


meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "Assign"
      input_arg {
        name: "ref"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "value"
        type_attr: "T"
      }
      output_arg {
        name: "output_ref"
        type_attr: "T"
        

In [5]:
import pickle
# get all trainable weights and save them in a dictionary
vars = sess.graph.get_collection('trainable_variables')
weights = {}
for v in vars:
    weights[v.name] = sess.run(v)  # retrieve the value from the tf backend

In [12]:
weights.keys()

dict_keys(['model/conv2d/kernel:0', 'model/conv2d/bias:0', 'model/conv2d_1/kernel:0', 'model/conv2d_1/bias:0', 'model/conv2d_2/kernel:0', 'model/conv2d_2/bias:0', 'model/conv2d_3/kernel:0', 'model/conv2d_3/bias:0', 'model/conv2d_4/kernel:0', 'model/conv2d_4/bias:0', 'model/dense/kernel:0', 'model/dense/bias:0', 'bidirectional_rnn/fw/lstm_cell/kernel:0', 'bidirectional_rnn/fw/lstm_cell/bias:0', 'bidirectional_rnn/bw/lstm_cell/kernel:0', 'bidirectional_rnn/bw/lstm_cell/bias:0', 'dense/kernel:0', 'dense/bias:0'])

In [ ]:
H W N C

In [13]:
for k, v in weights.items():
    print(k, v.shape)

model/conv2d/kernel:0 (5, 3, 1, 64)
model/conv2d/bias:0 (64,)
model/conv2d_1/kernel:0 (5, 1, 64, 128)
model/conv2d_1/bias:0 (128,)
model/conv2d_2/kernel:0 (5, 1, 128, 256)
model/conv2d_2/bias:0 (256,)
model/conv2d_3/kernel:0 (5, 1, 256, 512)
model/conv2d_3/bias:0 (512,)
model/conv2d_4/kernel:0 (5, 1, 512, 512)
model/conv2d_4/bias:0 (512,)
model/dense/kernel:0 (6144, 512)
model/dense/bias:0 (512,)
bidirectional_rnn/fw/lstm_cell/kernel:0 (640, 512)
bidirectional_rnn/fw/lstm_cell/bias:0 (512,)
bidirectional_rnn/bw/lstm_cell/kernel:0 (640, 512)
bidirectional_rnn/bw/lstm_cell/bias:0 (512,)
dense/kernel:0 (256, 1)
dense/bias:0 (1,)


In [11]:
def convert_tf1_to_tf2(checkpoint_path, output_prefix):
    vars = {}
    reader = tf.train.load_checkpoint(checkpoint_path)
    dtypes = reader.get_variable_to_dtype_map()
    for key in dtypes.keys():
        vars[key] = tf.Variable(reader.get_tensor(key))
    return tf.train.Checkpoint(vars=vars).save(output_prefix)

In [19]:
converted_path = convert_tf1_to_tf2('/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/tmp', '/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/tmp1/') 
print("\n[Converted]") 

TypeError: convert_tf1_to_tf2() missing 1 required positional argument: 'output_prefix'

In [15]:
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)

'/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/tmp1/-1'

In [20]:
model = tf.keras.models.load_model("/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/tmp")

OSError: SavedModel file does not exist at: /home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/tmp/{saved_model.pbtxt|saved_model.pb}

In [8]:
g = sess.graph
input_tensor = g.get_tensor_by_name("input:0")
output_tensor = g.get_tensor_by_name("output:0")

In [9]:
model = tf.keras.Model(input_tensor, output_tensor, name="tf2_model")
model.save("/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/tmp/tf2")

ValueError: Found unexpected instance while processing input tensors for keras functional model. Expecting KerasTensor which is from tf.keras.Input() or output from keras layer call(). Got: Tensor("input:0", shape=(None, 100, 3, 1), dtype=float32)

In [ ]:
saver = tf.train.Saver()
save_path = saver.save(sess, "./tmp/model.ckpt")
print("Model saved in path: %s" % save_path)

In [7]:
import onnx
import torch
from onnx2torch import convert
print(convert.__code__.co_varnames)
# # Path to ONNX model
# onnx_model_path = "/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/onnx_model/model.onnx"
# # You can pass the path to the onnx model to convert it or...
# torch_model_1 = convert(onnx_model_path)

('onnx_model_or_path', 'save_input_names', 'attach_onnx_mapping', 'onnx_model', 'opset_import', 'torch_graph', 'torch_initializers', 'torch_modules', 'input_value', 'name', 'placeholder_name', 'onnx_node', 'version', 'converter', 'torch_module', 'onnx_mapping', 'args', 'value_name', 'value_type', 'onnx_input_node', '_', 'torch_input_node', 'index', 'len_torch_initializers', 'torch_buffer_name', 'kwargs', 'first_skipped_arg', 'forward_args', 'kwargs_values', 'onnx_output_nodes', 'torch_output_nodes', 'torch_model')


In [ ]:
model = model.strip()
if model not in ['CHAP', 'CHAP_A', 'CHAP_B', 'CHAP_C', 'CHAP_ALL_ADULTS', 'CHAP_CHILDREN', 'CHAP_AUSDIAB', 'CUSTOM_MODEL']:
    raise Exception('model should be one of: CHAP, CHAP_A, CHAP_B, CHAP_C, CHAP_ALL_ADULTS, CHAP_CHILDREN, CHAP_AUSDIAB or CUSTOM_MODEL')

subject_ids = [fname.split('.')[0] for fname in os.listdir(pre_processed_data_dir) if not fname.startswith('.')]


perform_ensemble = False
if model == 'CHAP':
    models = ['CHAP_A', 'CHAP_B', 'CHAP_C']
    perform_ensemble = True
else:
    models = [model]

for model in models:
    if not os.path.exists(os.path.join(output_dir, '{}'.format(model))):
        os.makedirs(os.path.join(output_dir, '{}'.format(model)))

    tf.reset_default_graph()
    p = max(1, multiprocessing.cpu_count()//2)
    # sess = tf.Session(config=tf.ConfigProto(inter_op_parallelism_threads=p, intra_op_parallelism_threads=p))
    # tf.saved_model.loader.load(sess, ["serve"], os.path.join(model_ckpt_path, "{}".format(model)))


    for subject_id in subject_ids:
        if not args.silent:
            logger.info('Starting prediction generation for the subject {}'.format(subject_id))
        data = list(input_iterator(pre_processed_data_dir, subject_id))
        x, timestamps, labels = [d[0].reshape(-1, int(1/downsample_window * cnn_window_size),
                                        int(gt3x_frequency*downsample_window), 1) for d in data], [d[1] for d in data], [d[2] for d in data]
        fout = open(os.path.join(output_dir, "{}".format(model), "{}.csv".format(subject_id)), 'w')

        if segment:
            fout.write('segment,')
        fout.write('timestamp')
        if output_label:
            fout.write(',label')

        fout.write(',prediction\n')

        for n in range(len(x)):
            wanna_be = bi_lstm_window_sizes[model] * \
                        int(60*downsample_window)
            border = x[n].shape[0] % wanna_be
            wrapped = False
            zeroed = False
            if border != 0:
                if padding == "drop":
                    x[n] = x[n][:-border]
                    timestamps[n] = timestamps[n][:-border]
                    labels[n] = labels[n][:-border]
                    # deficit = wanna_be - border
                    # print("Dropped: {} sec".format((42 - deficit) * 10))


                else:
                    
                    deficit = wanna_be - border
                    increment = int(1 / downsample_window)
                    labels_padded = np.full(deficit, -1)

                    if padding == "zero":
                        x_padded = np.zeros(
                            [deficit] + list(x[n].shape[1:]))
                        timestamps_padded = np.full(
                            deficit,
                            timestamps[n][-1]
                        ) + np.array(
                            [increment * (i + 1) for i in range(deficit)])
                        x[n] = np.vstack((x[n], x_padded))
                        timestamps[n] = np.hstack(
                            (timestamps[n], timestamps_padded))
                        zeroed = True
                        
                    if padding == "wrap":
                        x_last_p1 = x[n][:-border]
                        x_last_p2 = x[n][-wanna_be:] 
                        x[n] = np.vstack((x_last_p1, x_last_p2))
                        wrapped = True
                    labels[n] = np.hstack((labels[n], labels_padded))

            y_pred = []
            for k in range(0, x[n].shape[0], bi_lstm_window_sizes[model] * int(60*downsample_window)):
                temp = x[n][k:k + bi_lstm_window_sizes[model] * int(60*downsample_window)]
                y_pred.append(sess.run('output:0', feed_dict={'input:0': temp}).flatten())
            y_pred = np.array(y_pred).flatten()
            
            if padding == "wrap" and wrapped:
                y_pred = np.hstack((y_pred[:-wanna_be], y_pred[-border:]))
            elif padding == "zero" and zeroed:
                y_pred = y_pred[:-deficit]


            for t, l, pred in zip(timestamps[n], labels[n], y_pred):
                formatstr = ""
                if segment:
                    formatstr += "{},{}"
                    values = [n, datetime.fromtimestamp(t).strftime("%Y-%m-%d %H:%M:%S")]
                else:
                    formatstr += "{}"
                    values = [datetime.fromtimestamp(t).strftime("%Y-%m-%d %H:%M:%S")]

                if output_label:
                    formatstr += ",{}"
                    values.append(label_map[int(l)])    

                formatstr += ",{}\n"
                values.append(label_map[int(pred)])

                fout.write(formatstr.format(*values))

        fout.close()
        if not args.silent:
            logger.info('Completed prediction generation for the subject {}'.format(subject_id))